In [2]:
from hashlib import sha256
import json
import time


#block

class block:
    def __init__(self, index, transaction, timestample, prev_hash):
        self.index = index
        self.transaction = transaction
        self.timestample = timestample
        self.prev_hash = prev_hash
        
    def compute_hash(self):
    
        block_string = json.dumps(self.__dict__, sort_keys=True)
        return sha256(block_string.encode()).hexdigest()

class transaction:
    def __init__(self, SN, from_key, to_key, validation):
        self.SN = SN
        self.from_key = from_key
        self.to_key = to_key
        self.validation = validation    


chain = dict()
chain['b0'] = block(0,transaction('sis',"gene","n",True),0,0)   #Genesis
chain_status = True



def send(from_key, to_key, SN):                   
    check_chain()
    if wallets[from_key].kind == 'plane' and wallets[to_key].kind == 'plane':  #se for de plane para plane requer intermediário
        if chain_status == True:
            if SN in wallets[from_key].parts:
                maint = input('What`s your Maintenance code?')
                wallets[from_key].parts.pop(wallets[from_key].parts.index(SN))
                wallets[to_key].parts.append(SN)
                print('part sent')        
                key = 'b{}'.format(len(chain))      #para recuperar os dados cache["ab123"].SN/from_key...
                chain[key] = block(len(chain),transaction(SN, wallets[from_key], wallets[maint], True),'time',hash(str(chain['b{}'.format(len(chain)-1)].transaction.from_key)+str(chain['b{}'.format(len(chain)-1)].transaction.SN)))        
                key = 'b{}'.format(len(chain))      
                chain[key] = block(len(chain),transaction(SN, wallets[maint], wallets[to_key], True),'time',hash(str(chain['b{}'.format(len(chain)-1)].transaction.from_key)+str(chain['b{}'.format(len(chain)-1)].transaction.SN)))        
                
                check_chain()
                if chain_status == False:
                    print('Transaction invalid, block deleted') #checa se a transação afetou a cadeia e exclui o bloco caso o tenha feito
                    del(chain['b{}'.format(len(chain)-1)])
            
            elif SN not in wallets[from_key].parts:
                print('Part not in your wallet')
    
    else:
        if chain_status == True:
            if SN in wallets[from_key].parts:
                wallets[from_key].parts.pop(wallets[from_key].parts.index(SN))
                wallets[to_key].parts.append(SN)
                print('part sent')        
                key = 'b{}'.format(len(chain))      #para recuperar os dados cache["ab123"].SN/from_key...
                chain[key] = block(len(chain),transaction(SN, wallets[from_key], wallets[to_key], True),'time',hash(str(chain['b{}'.format(len(chain)-1)].transaction.from_key)+str(chain['b{}'.format(len(chain)-1)].transaction.SN)))        
                check_chain()
                if chain_status == False:
                    print('Transaction invalid, block deleted') #checa se a transação afetou a cadeia e exclui o bloco caso o tenha feito
                    del(chain['b{}'.format(len(chain)-1)])
            elif SN not in from_key.parts:
                print('Part not in your wallet')
    
    
wallets = dict()

class wallet:
    def __init__(self,parts,public_key,private_key,kind):
        self.parts = parts
        self.public_key = public_key
        self.private_key = private_key
        self.kind = kind

    import random
    import string
    
    def new_wallet(kind): 
        import random
        import string 
        name = str(''.join(random.choices(string.ascii_uppercase + string.digits, k=8)))
        wallets[name] = wallet([],name,name + 'key',kind)
        print('Your wallet is:{}'.format(name))
    
def check_chain():
    global chain_status
    last_block = -1    
    
    while len(chain) + last_block != 0:
        previous_hash = hash(str(chain['b{}'.format(len(chain)+last_block-1)].transaction.from_key)+str(chain['b{}'.format(len(chain)+last_block-1)].transaction.SN))
        if chain['b{}'.format(len(chain)+last_block)].prev_hash == previous_hash:
            last_block = last_block - 1
            
        else:
            print ('Block {} invalid'.format(len(chain)+last_block))
            last_block = last_block - 1
            chain_status = False
        
    #criar variável "chain status" e não permitir transações quando estiver False
        
        
        
        #criar sistema de explorer
        #ex: hist_part(SN)
def hist_part(sn):
    last_block = -1    
    
    while len(chain) + last_block != 0:
        if sn == chain['b{}'.format(len(chain)+last_block)].transaction.SN:
            print ('from {} to {} at {}'.format((chain['b{}'.format(len(chain)+last_block)].transaction.from_key.public_key),(chain['b{}'.format(len(chain)+last_block)].transaction.to_key.public_key),(chain['b{}'.format(len(chain)+last_block)].timestample)))
            last_block = last_block - 1
        else:
            last_block = last_block - 1


In [3]:
wallets['a'] = wallet([1,2,3,4,5,6],'a','a1','provider')
wallets['b'] = wallet([],'b','b1','maint')
wallets['c'] = wallet([],'c','c1','maint')
wallets['d'] = wallet([],'d','d1','plane')
wallets['e'] = wallet([],'e','e1','plane')
wallets['f'] = wallet([],'f','f1','plane')
wallets['g'] = wallet([],'g','g1','maint')
wallets['h'] = wallet([],'h','h1','plane')
wallets['i'] = wallet([],'i','i1','maint')
wallets['j'] = wallet([],'j','j1','maint')
wallets['k'] = wallet([],'k','k1','maint')
wallets['l'] = wallet([],'l','l1','plane')
wallets['m'] = wallet([],'m','m1','plane')


In [4]:
send('a','b',1)
send('a','b',2)
send('a','c',3)
send('a','c',4)
send('b','d',1)
send('b','e',2)
send('c','f',3)
send('c','f',4)
send('f','i',3)
send('f','i',4)
send('d','g',1)
send('e','h',2)
send('h','j',2)
send('i','k',3)
send('i','k',4)
send('k','l',3)
send('k','m',4)

part sent
part sent
part sent
part sent
part sent
part sent
part sent
part sent
part sent
part sent
part sent
What`s your Maintenance code?k
part sent
part sent
part sent
part sent
part sent
part sent


In [5]:
chain

{'b0': <__main__.block at 0x7f7e86723e10>,
 'b1': <__main__.block at 0x7f7e866db1d0>,
 'b2': <__main__.block at 0x7f7e866db128>,
 'b3': <__main__.block at 0x7f7e866db208>,
 'b4': <__main__.block at 0x7f7e866db048>,
 'b5': <__main__.block at 0x7f7e866db6a0>,
 'b6': <__main__.block at 0x7f7e866db2b0>,
 'b7': <__main__.block at 0x7f7e866db4e0>,
 'b8': <__main__.block at 0x7f7e866db780>,
 'b9': <__main__.block at 0x7f7e866dbda0>,
 'b10': <__main__.block at 0x7f7e866dbcc0>,
 'b11': <__main__.block at 0x7f7e866dbc18>,
 'b12': <__main__.block at 0x7f7e866db828>,
 'b13': <__main__.block at 0x7f7e866dbb70>,
 'b14': <__main__.block at 0x7f7e866dbd68>,
 'b15': <__main__.block at 0x7f7e866db9b0>,
 'b16': <__main__.block at 0x7f7e866dba20>,
 'b17': <__main__.block at 0x7f7e866db7b8>,
 'b18': <__main__.block at 0x7f7e866dba90>}

In [6]:
send('d','e',1)

Part not in your wallet


In [9]:
chain['b10'].transaction.from_key

In [10]:
hist_part(1)

from d to g at time
from b to d at time
from a to b at time


In [2]:
wallet.new_wallet()

Your wallet is:T62M5TTQ


In [10]:
chain_status

True

In [32]:
check_chain()

In [60]:
type(input('bla'))

blaa


str

In [69]:
wallet.new_wallet('plane')

Your wallet is:V8DKNSNS


In [86]:
wallets['V8DKNSNS'].parts

123

In [83]:
wallets['UOLU4EZ4'].kind

'plane'

In [47]:
wallets

{'a': <__main__.wallet at 0x7f28203d3278>,
 'b': <__main__.wallet at 0x7f28203d3240>,
 'c': <__main__.wallet at 0x7f282041ca90>,
 'd': <__main__.wallet at 0x7f28203d3320>,
 'e': <__main__.wallet at 0x7f28203d3390>}

{'b0': <__main__.block at 0x7fd86824e630>}